## Przykładowy montaż filmu

In [2]:
from moviepy.editor import *
from os import listdir
from os.path import isfile, join
from moviepy.video.tools.segmenting import findObjects
import numpy as np

In [3]:
#Załadowanie wszystkich klipów z folderu, warto znać ich kolejność
clip_path = "my_street_film/clips/"
files = [f for f in listdir(clip_path)]
files = [clip_path + f for f in files]
i=0
for n in files:
    print(i,":",n)
    i+=1
multi_clips = [VideoFileClip(n) for n in files]

0 : my_street_film/clips/MVI_7176.AVI
1 : my_street_film/clips/MVI_7177.AVI
2 : my_street_film/clips/MVI_7179.AVI
3 : my_street_film/clips/MVI_7182.AVI
4 : my_street_film/clips/MVI_7183.AVI
5 : my_street_film/clips/MVI_7184.AVI
6 : my_street_film/clips/MVI_7185.AVI
7 : my_street_film/clips/MVI_7186.AVI
8 : my_street_film/clips/MVI_7188.AVI
9 : my_street_film/clips/MVI_7190.AVI
10 : my_street_film/clips/MVI_7191.AVI
11 : my_street_film/clips/MVI_7197.AVI
12 : my_street_film/clips/MVI_7198.AVI
13 : my_street_film/clips/MVI_7202.AVI
14 : my_street_film/clips/MVI_7205.AVI


In [14]:
scene1_road = multi_clips[5].subclip(2,4).set_start(0)
scene1_lantern = multi_clips[10].subclip(6,8).set_start(scene1_road.end)
scene1_grass = multi_clips[9].subclip(28,30).set_start(scene1_lantern.end)

title = TextClip('my_street', font='Consolas', color='white', fontsize= 70)
title = title.set_duration(3).set_start(scene1_grass.end).set_position("center")

title2 = TextClip('by bartlomiej kopysc', font='Consolas', color='white', fontsize=20)
title2 = title2.set_duration(3).set_start(scene1_grass.end).set_position((0.33,0.7), relative=True)

scene2_enter = multi_clips[0].subclip(0,7).set_start(title.end)
scene2_block = multi_clips[1].subclip(2,5).set_start(scene2_enter.end)
scene2_block = scene2_block.set_audio(scene2_block.audio.volumex(0.5))
scene2_path = multi_clips[7].subclip(3,6).set_start(scene2_block.end)

scene3_trees = multi_clips[2].subclip(6,10).set_start(scene2_path.end)
scene3_treeclose = multi_clips[3].set_duration(6).set_start(scene3_trees.end)
eff1_grass = multi_clips[9].subclip(26,32)
eff1_cars = multi_clips[8].subclip(14,20)

eff2_dirt = multi_clips[12].subclip(2,9)
eff2_candles = multi_clips[13].subclip(2,9)

eff3_road = multi_clips[7].subclip(2,7).volumex(0.5)
eff3_lamp = multi_clips[10].subclip(6,11).volumex(0.5)
eff3_candle = multi_clips[14].subclip(0,5).volumex(0.5)
eff3_trees = multi_clips[2].subclip(6,11).volumex(0.5)

#ciekawy efekt1
mask1 = ImageClip("my_street_film/images/mask2.png")
eff_clips = [eff1_cars,eff1_grass]
regions = findObjects(mask1)
compose_clips =  [c.resize(r.size)
                   .set_mask(r.mask)
                   .set_pos(r.screenpos)
                  for c,r in zip(eff_clips,regions)]

effect_clip1 = CompositeVideoClip(compose_clips,mask1.size)
effect_clip1 = effect_clip1.set_start(scene3_treeclose.end)
#Koniec efektu

#ciekawy efekt2
mask2 = ImageClip("my_street_film/images/mask3b.png")
eff_clips = [eff2_candles,eff2_dirt]
regions = findObjects(mask2)
compose_clips2 =  [c.resize(r.size)
                   .set_mask(r.mask)
                   .set_pos(r.screenpos)
                  for c,r in zip(eff_clips,regions)]

effect_clip2 = CompositeVideoClip(compose_clips2,mask2.size)
effect_clip2 = effect_clip2.set_start(effect_clip1.end)
#Koniec efektu

#ciekawy efekt3
mask3 = ImageClip("my_street_film/images/mask4.png")
eff_clips = [eff3_candle, eff3_lamp, eff3_road, eff3_trees]
regions = findObjects(mask3)
compose_clips3 =  [c.resize(r.size)
                   .set_mask(r.mask)
                   .set_pos(r.screenpos)
                  for c,r in zip(eff_clips,regions)]

effect_clip3 = CompositeVideoClip(compose_clips3,mask3.size)
effect_clip3 = effect_clip3.set_start(effect_clip2.end)
#Koniec efektu

scene4_candleclose = multi_clips[14].subclip(0,4).set_start(effect_clip3.end)
scene4_candleslow = scene4_candleclose.fl_time(lambda t: 0.7*t).set_duration(8).fadeout(2)

title_end = TextClip("made with moviepy\n2021", font='Consolas', color='white', fontsize = 30)
title_end = title_end.set_duration(6).set_start(scene4_candleslow.end).set_position("center").fadein(1)


# Łączenie wszystkich klipów

music1 = AudioFileClip('my_street_film/music/Spring Migration - The Great North Sound Society.mp3')
music1 = music1.set_start(scene1_grass.start).set_end(title_end.end)

final_clips = [scene1_road, scene1_lantern,scene1_grass,scene2_block,scene2_enter,scene2_path,
               scene3_treeclose,scene3_trees,scene4_candleslow,effect_clip1, effect_clip2, effect_clip3,
              title, title2, title_end]

final = CompositeVideoClip(final_clips)
final_audio = CompositeAudioClip([final.audio, music1])
final_audio = final_audio.audio_fadeout(6)
final = final.set_audio(final_audio)


In [ ]:
#zapis do pliku
final.write_videofile('my_street_film/film.mp4', bitrate = "8000k")